In [310]:
alldev = !ls /sigmorphon2020/task0-data/*/*.dev
alltrn = !ls /sigmorphon2020/task0-data/*/*.trn

In [311]:
alldata = alldev+alltrn

In [312]:
import shared.unimorph_loader.provided as unl
import os.path
import pandas as pn

In [313]:
everything = [(os.path.basename(i),unl.read_unimorph_tsv(i)) for i in alldata[:2]]

In [314]:
all_lemmata = list()
for langname, lang in everything:
    for lem in lang["lemma"]:
        all_lemmata.append(lem)

In [315]:
len(all_lemmata)

187

In [316]:
import shared.unimorph_loader.alphabets as alpha
from itertools import count
from collections import OrderedDict

In [317]:
my_alpha = alpha.get_master_alphabet()
my_alpha_lookup = OrderedDict(zip(my_alpha,count()))
START_TOK = my_alpha[1]
END_TOK = my_alpha[0]

In [318]:
all_converted_lemmata = [ [1] + [my_alpha_lookup[j] for j in lem.lower()] + [0] for lem in all_lemmata]

In [319]:
def lem_seq_to_onehot(one_seq,alph_len):
    one_hot_version = torch.zeros(one_seq.shape[0],1,alph_len)
    one_hot_version[:,0,one_seq[:,0]] = 1.0
    return one_hot_version

In [320]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.utils.data

torch.set_default_dtype(torch.double)
torch.manual_seed(1)

In [321]:
import torch.nn.utils.rnn as rnn_utils

In [322]:
class MyAuto(torch.nn.Module):
    def __init__(self,an_alphabet):
        super(MyAuto,self).__init__()
        self.N_hidden = 300
        self.D_embed = 200
        self.embedding = nn.Embedding(len(an_alphabet),self.D_embed)
        self.encoder = nn.RNN(self.D_embed,self.N_hidden,2,dropout=0.1)
        #self.encoder = nn.Identity()
        self.d1 = nn.Linear(self.N_hidden,len(an_alphabet))
        self.d2 = nn.Softmax(dim=-1)
    
    def forward(self, some_data):
        is_packed = isinstance(some_data,rnn_utils.PackedSequence)
        
        if is_packed:
            embedded_data = self.embedding(some_data.data)
        else:
            embedded_data = self.embedding(some_data)
        outputs,hidden = self.encoder(embedded_data)
        #outputs = self.encoder(embedded_data)
        x = self.d1(outputs)
        y = self.d2(x)
        if is_packed:
            return rnn_utils.PackedSequence(y,some_data.batch_sizes,some_data.sorted_indices, some_data.unsorted_indices)
        else:
            return y
        

In [334]:
class MyAuto(torch.nn.Module):
    def __init__(self,an_alphabet):
        super(MyAuto,self).__init__()
        self.N_hidden = 300
        self.D_embed = 20
        self.embedding = nn.Embedding(len(an_alphabet),self.D_embed)
        self.d1 = nn.Linear(self.D_embed,len(an_alphabet))
        self.d2 = nn.Softmax(dim=-1)
    
    def forward(self, some_data):
        is_packed = isinstance(some_data,rnn_utils.PackedSequence)
        
        if is_packed:
            embedded_data = self.embedding(some_data.data)
        else:
            embedded_data = self.embedding(some_data)
        #outputs,hidden = self.encoder(embedded_data)
        
        x = self.d1(embedded_data)
        y = self.d2(x)
        if is_packed:
            return rnn_utils.PackedSequence(y,some_data.batch_sizes,some_data.sorted_indices, some_data.unsorted_indices)
        else:
            return y

In [335]:
def tensor_seq_to_onehot(s,nlabels=len(my_alpha)):
    bar = torch.zeros(s.shape[0],1,nlabels)
    bar.scatter_(-1,s.view(-1,1,1),1)
    return bar

In [336]:
all_t_lemmata = [torch.LongTensor(i).reshape(-1,1) for i in all_converted_lemmata]

In [337]:
all_oh_lemmata = [tensor_seq_to_onehot(i) for i in all_t_lemmata]

In [338]:
len(all_t_lemmata)

187

In [339]:
model = MyAuto(my_alpha)
model.zero_grad()

In [340]:
myloss = nn.MSELoss()

In [341]:

opt = torch.optim.SGD(model.parameters(),lr=0.1)

In [342]:
for i in range(10000):
    model.zero_grad()
    
    for j in range(10,len(all_t_lemmata),10):
        
        packed_x = rnn_utils.pack_sequence(all_t_lemmata[j-10:j],enforce_sorted=False)
        packed_y = rnn_utils.pack_sequence(all_oh_lemmata[j-10:j],enforce_sorted=False)
        
        pred = model(packed_x)
        l = myloss(pred.data,packed_y.data)
        l.backward()
        
        opt.step()
        opt.zero_grad()
    
    if (i%200==0):
        print(i)
        with torch.no_grad():
            total_loss = torch.tensor([0.0])
            
            for j in range(len(all_t_lemmata)):
                pred = model(all_t_lemmata[j])
                l = myloss(pred,all_oh_lemmata[j])
                total_loss += l
            print(total_loss)
    

0
tensor([0.8312])
200
tensor([0.8309])
400
tensor([0.8307])
600
tensor([0.8304])
800
tensor([0.8300])
1000
tensor([0.8296])
1200
tensor([0.8292])
1400
tensor([0.8287])
1600
tensor([0.8281])
1800
tensor([0.8273])
2000
tensor([0.8264])
2200
tensor([0.8252])
2400
tensor([0.8235])
2600
tensor([0.8209])
2800
tensor([0.8166])
3000
tensor([0.8087])
3200
tensor([0.7939])
3400
tensor([0.7765])
3600
tensor([0.7668])
3800
tensor([0.7626])
4000
tensor([0.7605])
4200
tensor([0.7591])
4400
tensor([0.7580])
4600
tensor([0.7571])
4800
tensor([0.7562])
5000
tensor([0.7554])
5200
tensor([0.7544])
5400
tensor([0.7534])
5600
tensor([0.7522])
5800
tensor([0.7508])
6000
tensor([0.7492])
6200
tensor([0.7471])


KeyboardInterrupt: 

In [218]:
model(all_t_lemmata[0]).argmax(-1)

tensor([[177],
        [177],
        [101],
        [140],
        [ 42],
        [163],
        [ 91],
        [  7],
        [  7],
        [217]])

In [304]:
a = rnn_utils.pad_sequence(all_t_lemmata[:10])
b = rnn_utils.pack_sequence(all_t_lemmata[:10],enforce_sorted=False)
by = rnn_utils.pack_sequence(all_oh_lemmata[:10],enforce_sorted=False)

In [307]:
myloss(model(b).data,by.data)

tensor(0.0044, grad_fn=<MseLossBackward>)

In [286]:
model.embedding(b.data).shape

torch.Size([104, 1, 200])

In [287]:
b.batch_sizes

tensor([10, 10, 10, 10, 10, 10, 10, 10,  9,  8,  5,  2])

In [288]:
b

PackedSequence(data=tensor([[ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [44],
        [44],
        [44],
        [44],
        [44],
        [44],
        [44],
        [44],
        [44],
        [44],
        [46],
        [46],
        [46],
        [46],
        [46],
        [46],
        [46],
        [46],
        [46],
        [46],
        [47],
        [39],
        [47],
        [33],
        [47],
        [47],
        [47],
        [44],
        [38],
        [42],
        [32],
        [40],
        [32],
        [32],
        [32],
        [32],
        [32],
        [32],
        [52],
        [52],
        [51],
        [45],
        [33],
        [43],
        [38],
        [51],
        [51],
        [45],
        [45],
        [39],
        [40],
        [52],
        [40],
        [40],
        [32],
        [32],
        [32],
        [36],
        [40],
        [32],


In [289]:
rnn_utils.PackedSequence

torch.nn.utils.rnn.PackedSequence

In [290]:
b.sorted_indices

tensor([1, 6, 2, 7, 8, 0, 5, 9, 4, 3])

In [291]:
b.unsorted_indices

tensor([5, 0, 2, 9, 8, 6, 1, 3, 4, 7])